In [1]:
# install geopy & folium
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
# import Libraries that we need
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [9]:
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

## Foursquare Credentials 

In [3]:
ClIENT_ID = '5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4' # your Foursquare ID
ClIENT_SECRET = '0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: 5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4
Foursquare_Secret:0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ


## Latitude/Longitude

In [4]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


## Search for Parks

In [5]:
# search for parks
search_query = 'Park'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4&client_secret=0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ&ll=43.653963,-79.387207&v=20180604&query=Park&radius=500&limit=30'

## get resulsts as JSON

In [10]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

## Create Dataframe

In [11]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dfParks = json_normalize(venues)
dfParks.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b54deadf964a520a3d027e3,NaN,CA,Toronto,Canada,NaN,423,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,NaN,ON,Grange Park,v-1565154671,NaN
1,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4ad80fcef964a520a61021e3,280 Simcoe St,CA,Toronto,Canada,Dundas Street West,282,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,NaN,M5T 2Y5,ON,One Park Lane,v-1565154671,NaN
2,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",False,4b9177e4f964a520d2be33e3,NaN,CA,Toronto,Canada,NaN,360,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.650735, 'lng':...",43.650735,-79.386921,NaN,NaN,ON,TTC Streetcar #403 - Victoria park,v-1565154671,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4fbab8dbe4b0f1ce5b6858bb,130 Queen St W,CA,Toronto,Canada,at University Ave,327,"[130 Queen St W (at University Ave), Toronto O...","[{'label': 'display', 'lat': 43.65110457967368...",43.651105,-79.386269,NaN,M5H 2N5,ON,Osgoode Hall Park,v-1565154671,NaN
4,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4faac3d5e4b06c670e1afcc4,424 Yonge,CA,Toronto,Canada,College and Yonge,114,"[424 Yonge (College and Yonge), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65293604935704...",43.652936,-79.387083,NaN,NaN,ON,College Park Residences,v-1565154671,NaN


## Clean Dataframe

In [12]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in dfParks.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = dfParks.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Grange Park,Park,NaN,CA,Toronto,Canada,NaN,423,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,NaN,ON,4b54deadf964a520a3d027e3
1,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,CA,Toronto,Canada,Dundas Street West,282,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,NaN,M5T 2Y5,ON,4ad80fcef964a520a61021e3
2,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,CA,Toronto,Canada,NaN,360,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.650735, 'lng':...",43.650735,-79.386921,NaN,NaN,ON,4b9177e4f964a520d2be33e3
3,Osgoode Hall Park,Park,130 Queen St W,CA,Toronto,Canada,at University Ave,327,"[130 Queen St W (at University Ave), Toronto O...","[{'label': 'display', 'lat': 43.65110457967368...",43.651105,-79.386269,NaN,M5H 2N5,ON,4fbab8dbe4b0f1ce5b6858bb
4,College Park Residences,Residential Building (Apartment / Condo),424 Yonge,CA,Toronto,Canada,College and Yonge,114,"[424 Yonge (College and Yonge), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65293604935704...",43.652936,-79.387083,NaN,NaN,ON,4faac3d5e4b06c670e1afcc4


In [13]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','address','postalCode','neighborhood', 'id'], axis=1)
clean_park_dataframe2

,name,categories,lat,lng,state
0,Grange Park,Park,43.652488,-79.392053,ON
1,One Park Lane,Residential Building (Apartment / Condo),43.655442,-79.390056,ON
2,TTC Streetcar #403 - Victoria park,Light Rail Station,43.650735,-79.386921,ON
3,Osgoode Hall Park,Park,43.651105,-79.386269,ON
4,College Park Residences,Residential Building (Apartment / Condo),43.652936,-79.387083,ON
5,Larry Sefton Park,Plaza,43.654589,-79.383405,NaN
6,Trinity Park And Labyrinth,Garden,43.654224,-79.381882,ON
7,Butterfield Park,Park,43.652697,-79.391091,ON
8,One Park Lane,Residential Building (Apartment / Condo),43.656271,-79.390256,ON
9,Trinity Square Park,Plaza,43.654379,-79.381849,ON


In [14]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,lat,lng,state
0,Grange Park,Park,43.652488,-79.392053,ON
1,One Park Lane,Residential Building (Apartment / Condo),43.655442,-79.390056,ON
2,TTC Streetcar #403 - Victoria park,Light Rail Station,43.650735,-79.386921,ON
3,Osgoode Hall Park,Park,43.651105,-79.386269,ON
4,College Park Residences,Residential Building (Apartment / Condo),43.652936,-79.387083,ON
6,Trinity Park And Labyrinth,Garden,43.654224,-79.381882,ON
7,Butterfield Park,Park,43.652697,-79.391091,ON
8,One Park Lane,Residential Building (Apartment / Condo),43.656271,-79.390256,ON
9,Trinity Square Park,Plaza,43.654379,-79.381849,ON
11,Queens Park Ballroom,Auditorium,43.649458,-79.386126,ON


In [15]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,lat,lng,state
0,Grange Park,Park,43.652488,-79.392053,ON
3,Osgoode Hall Park,Park,43.651105,-79.386269,ON
7,Butterfield Park,Park,43.652697,-79.391091,ON


## Search for Event Space

In [16]:
# search for parks
search_query = 'Event Space'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4&client_secret=0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ&ll=43.653963,-79.387207&v=20180604&query=Event Space&radius=500&limit=30'

In [17]:
#Send the GET Request and examine the results
eresults = requests.get(url).json()
#cresults

In [18]:
# assign relevant part of JSON to venues
venues2 = eresults['response']['venues']

# tranform venues into a dataframe
Events_dataframe = json_normalize(venues2)
Events_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",False,4ad8cd16f964a520c91421e3,1 Dundas St E,CA,Toronto,Canada,at Yonge St,588,"[1 Dundas St E (at Yonge St), Toronto ON M5B 2...","[{'label': 'display', 'lat': 43.65605389742188...",43.656054,-79.380495,M5B 2R8,ON,Yonge-Dundas Square,v-1565154677,68861986
1,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",False,5a3269a22c7eb93dcc028bba,2464 Yonge St,CA,Toronto,Canada,NaN,206,"[2464 Yonge St, Toronto ON M4P 2H5, Canada]","[{'label': 'display', 'lat': 43.65464583327517...",43.654646,-79.384825,M4P 2H5,ON,Main Event Music,v-1565154677,NaN
2,[],False,4f36fba6e4b0b0eb8cbe105e,NaN,CA,Toronto,Canada,NaN,246,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65237594628107...",43.652376,-79.385082,NaN,ON,White Space,v-1565154677,NaN
3,"[{'id': '4bf58dd8d48988d100941735', 'name': 'M...",False,599d96f78194fc5c6dd8d545,600 Bay St,CA,Toronto,Canada,Bay & Dundas,323,"[600 Bay St (Bay & Dundas), Toronto ON M5G 1M6...","[{'label': 'display', 'lat': 43.655833, 'lng':...",43.655833,-79.384137,M5G 1M6,ON,Now Common • Space 306,v-1565154677,NaN
4,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...",False,4cd58f43dfb4a1cdf6e3595c,2 murray street,CA,Toronto,Canada,NaN,410,"[2 murray street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65682507705976...",43.656825,-79.390415,NaN,ON,The Toy Space,v-1565154677,NaN


## Clean Events Dataframe

In [19]:
# keep only columns that include venue name, and anything that is associated with location
Events_clean_columns = ['name', 'categories'] + [col for col in Events_dataframe.columns if col.startswith('location.')]+ ['id']
clean_events_dataframe = Events_dataframe.loc[:,Events_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_events_dataframe['categories'] = clean_events_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_events_dataframe.columns = [column.split('.')[-1] for column in clean_events_dataframe.columns]

clean_events_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Yonge-Dundas Square,Plaza,1 Dundas St E,CA,Toronto,Canada,at Yonge St,588,"[1 Dundas St E (at Yonge St), Toronto ON M5B 2...","[{'label': 'display', 'lat': 43.65605389742188...",43.656054,-79.380495,M5B 2R8,ON,4ad8cd16f964a520c91421e3
1,Main Event Music,Music Venue,2464 Yonge St,CA,Toronto,Canada,NaN,206,"[2464 Yonge St, Toronto ON M4P 2H5, Canada]","[{'label': 'display', 'lat': 43.65464583327517...",43.654646,-79.384825,M4P 2H5,ON,5a3269a22c7eb93dcc028bba
2,White Space,None,NaN,CA,Toronto,Canada,NaN,246,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65237594628107...",43.652376,-79.385082,NaN,ON,4f36fba6e4b0b0eb8cbe105e
3,Now Common • Space 306,Meeting Room,600 Bay St,CA,Toronto,Canada,Bay & Dundas,323,"[600 Bay St (Bay & Dundas), Toronto ON M5G 1M6...","[{'label': 'display', 'lat': 43.655833, 'lng':...",43.655833,-79.384137,M5G 1M6,ON,599d96f78194fc5c6dd8d545
4,The Toy Space,Toy / Game Store,2 murray street,CA,Toronto,Canada,NaN,410,"[2 murray street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65682507705976...",43.656825,-79.390415,NaN,ON,4cd58f43dfb4a1cdf6e3595c


In [20]:
# delete unnecessary columns
clean_events_dataframe2= clean_events_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','address','postalCode', 'id'], axis=1)
clean_events_dataframe2

,name,categories,lat,lng,state
0,Yonge-Dundas Square,Plaza,43.656054,-79.380495,ON
1,Main Event Music,Music Venue,43.654646,-79.384825,ON
2,White Space,None,43.652376,-79.385082,ON
3,Now Common • Space 306,Meeting Room,43.655833,-79.384137,ON
4,The Toy Space,Toy / Game Store,43.656825,-79.390415,ON
5,Your Space,Salon / Barbershop,43.654378,-79.380990,ON
6,Digital Futures Work Space,College Lab,43.648893,-79.389002,ON
7,Space,General Entertainment,43.649850,-79.390107,ON
8,OCADU sLab & SFI Collaboration Space,College Lab,43.649275,-79.389083,ON
9,eat:Strategy Conference & Events,Event Space,43.654053,-79.391945,ON


In [21]:
# delete rows with none values
df_events = clean_events_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_events

,name,categories,lat,lng,state
0,Yonge-Dundas Square,Plaza,43.656054,-79.380495,ON
1,Main Event Music,Music Venue,43.654646,-79.384825,ON
3,Now Common • Space 306,Meeting Room,43.655833,-79.384137,ON
4,The Toy Space,Toy / Game Store,43.656825,-79.390415,ON
5,Your Space,Salon / Barbershop,43.654378,-79.380990,ON
6,Digital Futures Work Space,College Lab,43.648893,-79.389002,ON
7,Space,General Entertainment,43.649850,-79.390107,ON
8,OCADU sLab & SFI Collaboration Space,College Lab,43.649275,-79.389083,ON
9,eat:Strategy Conference & Events,Event Space,43.654053,-79.391945,ON
10,GlocalSpace Inc.,Tech Startup,43.653925,-79.380639,ON


In [22]:
df_events1= df_events[df_events['categories']=='Event Space']
df_events1

,name,categories,lat,lng,state
9,eat:Strategy Conference & Events,Event Space,43.654053,-79.391945,ON
12,The Fifth Events,Event Space,43.649250,-79.389534,ON


## Search for Libraries 

In [23]:
# search for parks
search_query = 'Library'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4&client_secret=0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ&ll=43.653963,-79.387207&v=20180604&query=Library&radius=500&limit=30'

In [24]:
# Send the GET Request and examine the results
Lresults = requests.get(url).json()
#Rresults

In [25]:
# assign relevant part of JSON to venues
venues3 = Lresults['response']['venues']

# tranform venues into a dataframe
Library_dataframe = json_normalize(venues3)
Library_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4c8938c8944e224b52e72285,100 Queen St W,CA,Toronto,Canada,City Hall,348,"[100 Queen St W (City Hall), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65263139770985...",43.652631,-79.383295,M5H 2N3,ON,Toronto Public Library,v-1565154711
1,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4bf1c17424f19c74b073f983,130 Queen St W,CA,Toronto,Canada,at University Ave,269,"[130 Queen St W (at University Ave), Toronto O...","[{'label': 'display', 'lat': 43.651884, 'lng':...",43.651884,-79.385499,NaN,ON,LSUC Library,v-1565154711
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5a6b737b35f98359eed11974,281 Dundas St West,CA,Toronto,Canada,St Patrick And Dundas St W,301,[281 Dundas St West (St Patrick And Dundas St ...,"[{'label': 'display', 'lat': 43.65441282740799...",43.654413,-79.390902,M5T 2W5,ON,The Library Specialty Coffee,v-1565154711
3,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",False,4d375963036c54812b634c89,113 McCaul Street,CA,Toronto,Canada,NaN,198,"[113 McCaul Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65353807405525...",43.653538,-79.389607,NaN,ON,Dorothy H. Hoover Library - OCAD University,v-1565154711
4,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4f06065b02d5265073806d8f,317 Dundas St W,CA,Toronto,Canada,NaN,482,"[317 Dundas St W, Toronto ON M5T 1G4, Canada]","[{'label': 'display', 'lat': 43.65431693742484...",43.654317,-79.393175,M5T 1G4,ON,E.P. Taylor Research Library & Archives (AGO),v-1565154711


## Clean Library Dataframe

In [26]:
# keep only columns that include venue name, and anything that is associated with location
Library_clean_columns = ['name', 'categories'] + [col for col in Library_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Library_dataframe = Library_dataframe.loc[:,Library_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Library_dataframe['categories'] = clean_Library_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Library_dataframe.columns = [column.split('.')[-1] for column in clean_Library_dataframe.columns]

clean_Library_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Toronto Public Library,Library,100 Queen St W,CA,Toronto,Canada,City Hall,348,"[100 Queen St W (City Hall), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65263139770985...",43.652631,-79.383295,M5H 2N3,ON,4c8938c8944e224b52e72285
1,LSUC Library,College Library,130 Queen St W,CA,Toronto,Canada,at University Ave,269,"[130 Queen St W (at University Ave), Toronto O...","[{'label': 'display', 'lat': 43.651884, 'lng':...",43.651884,-79.385499,NaN,ON,4bf1c17424f19c74b073f983
2,The Library Specialty Coffee,Coffee Shop,281 Dundas St West,CA,Toronto,Canada,St Patrick And Dundas St W,301,[281 Dundas St West (St Patrick And Dundas St ...,"[{'label': 'display', 'lat': 43.65441282740799...",43.654413,-79.390902,M5T 2W5,ON,5a6b737b35f98359eed11974
3,Dorothy H. Hoover Library - OCAD University,College Academic Building,113 McCaul Street,CA,Toronto,Canada,NaN,198,"[113 McCaul Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65353807405525...",43.653538,-79.389607,NaN,ON,4d375963036c54812b634c89
4,E.P. Taylor Research Library & Archives (AGO),Library,317 Dundas St W,CA,Toronto,Canada,NaN,482,"[317 Dundas St W, Toronto ON M5T 1G4, Canada]","[{'label': 'display', 'lat': 43.65431693742484...",43.654317,-79.393175,M5T 1G4,ON,4f06065b02d5265073806d8f


In [27]:
# delete unnecessary columns
clean_Library_dataframe2= clean_Library_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                         'labeledLatLngs','address','postalCode', 'id'], axis=1)
clean_Library_dataframe2

,name,categories,lat,lng,state
0,Toronto Public Library,Library,43.652631,-79.383295,ON
1,LSUC Library,College Library,43.651884,-79.385499,ON
2,The Library Specialty Coffee,Coffee Shop,43.654413,-79.390902,ON
3,Dorothy H. Hoover Library - OCAD University,College Academic Building,43.653538,-79.389607,ON
4,E.P. Taylor Research Library & Archives (AGO),Library,43.654317,-79.393175,ON


In [28]:
# delete rows with none values
df_Library = clean_Library_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Library

,name,categories,lat,lng,state
0,Toronto Public Library,Library,43.652631,-79.383295,ON
1,LSUC Library,College Library,43.651884,-79.385499,ON
2,The Library Specialty Coffee,Coffee Shop,43.654413,-79.390902,ON
3,Dorothy H. Hoover Library - OCAD University,College Academic Building,43.653538,-79.389607,ON
4,E.P. Taylor Research Library & Archives (AGO),Library,43.654317,-79.393175,ON


## Search for Univerities 

In [29]:
# search for Cafeteria
search_query = 'University'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4&client_secret=0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ&ll=43.653963,-79.387207&v=20180604&query=University&radius=10000&limit=30'

In [30]:
# Send the GET Request and examine the results
Uresults = requests.get(url).json()
#cresults

In [31]:
# assign relevant part of JSON to venues
venues4 = Uresults['response']['venues']

# tranform venues into a dataframe
University_dataframe = json_normalize(venues4)
University_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",False,4ad4c05ef964a52097f620e3,27 King's College Cir,CA,Toronto,Canada,at King's College Rd,1148,"[27 King's College Cir (at King's College Rd),...","[{'label': 'display', 'lat': 43.6624934706167,...",43.662493,-79.395220,M5S 1A1,ON,University of Toronto,v-1565154716,57112483
1,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",False,4e6565fb1850e238d7048495,University Avenue,CA,Toronto,Canada,btwn Richmond & College,141,"[University Avenue (btwn Richmond & College), ...","[{'label': 'display', 'lat': 43.65269470686765...",43.652695,-79.387218,NaN,ON,University Avenue Plaza,v-1565154716,NaN
2,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4b13d2cff964a5207f9923e3,480 University Ave,CA,Toronto,Canada,Dundas,171,"[480 University Ave (Dundas), Toronto ON M5G 1...","[{'label': 'display', 'lat': 43.65509931314145...",43.655099,-79.388648,M5G 1V2,ON,480 University Ave,v-1565154716,NaN
3,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4cd044c29d87224bf129543b,NaN,CA,NaN,Canada,NaN,47,[Canada],"[{'label': 'display', 'lat': 43.653571, 'lng':...",43.653571,-79.386979,NaN,NaN,University Centre Gym,v-1565154716,NaN
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4c45c6c6da2176b09543637a,393 University Ave.,CA,Toronto,Canada,at Armoury St.,36,"[393 University Ave. (at Armoury St.), Toronto...","[{'label': 'display', 'lat': 43.65390719043468...",43.653907,-79.386764,M7A 2K1,ON,University Centre,v-1565154716,NaN


## Clean University Dataframe

In [32]:
# keep only columns that include venue name, and anything that is associated with location
University_clean_columns = ['name', 'categories'] + [col for col in University_dataframe.columns if col.startswith('location.')]+ ['id']
clean_University_dataframe = University_dataframe.loc[:,University_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_University_dataframe['categories'] = clean_University_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_University_dataframe.columns = [column.split('.')[-1] for column in clean_University_dataframe.columns]

clean_University_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,University of Toronto,University,27 King's College Cir,CA,Toronto,Canada,at King's College Rd,1148,"[27 King's College Cir (at King's College Rd),...","[{'label': 'display', 'lat': 43.6624934706167,...",43.662493,-79.395220,M5S 1A1,ON,4ad4c05ef964a52097f620e3
1,University Avenue Plaza,Plaza,University Avenue,CA,Toronto,Canada,btwn Richmond & College,141,"[University Avenue (btwn Richmond & College), ...","[{'label': 'display', 'lat': 43.65269470686765...",43.652695,-79.387218,NaN,ON,4e6565fb1850e238d7048495
2,480 University Ave,Office,480 University Ave,CA,Toronto,Canada,Dundas,171,"[480 University Ave (Dundas), Toronto ON M5G 1...","[{'label': 'display', 'lat': 43.65509931314145...",43.655099,-79.388648,M5G 1V2,ON,4b13d2cff964a5207f9923e3
3,University Centre Gym,Gym / Fitness Center,NaN,CA,NaN,Canada,NaN,47,[Canada],"[{'label': 'display', 'lat': 43.653571, 'lng':...",43.653571,-79.386979,NaN,NaN,4cd044c29d87224bf129543b
4,University Centre,Office,393 University Ave.,CA,Toronto,Canada,at Armoury St.,36,"[393 University Ave. (at Armoury St.), Toronto...","[{'label': 'display', 'lat': 43.65390719043468...",43.653907,-79.386764,M7A 2K1,ON,4c45c6c6da2176b09543637a


In [33]:
# delete unnecessary columns
clean_University_dataframe2= clean_University_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','address','postalCode', 'id'], axis=1)
clean_University_dataframe2

,name,categories,lat,lng,state
0,University of Toronto,University,43.662493,-79.395220,ON
1,University Avenue Plaza,Plaza,43.652695,-79.387218,ON
2,480 University Ave,Office,43.655099,-79.388648,ON
3,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,NaN
4,University Centre,Office,43.653907,-79.386764,ON
5,Ontario College of Art and Design University (...,University,43.652803,-79.391074,ON
6,438 University Ave,Office,43.654296,-79.388415,ON
7,505 University Ave.,Government Building,43.656177,-79.388549,ON
8,393 University Ave,Office,43.653752,-79.388150,ON
9,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,ON


In [34]:
# delete rows with none values
df_University = clean_University_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_University

,name,categories,lat,lng,state
0,University of Toronto,University,43.662493,-79.395220,ON
1,University Avenue Plaza,Plaza,43.652695,-79.387218,ON
2,480 University Ave,Office,43.655099,-79.388648,ON
4,University Centre,Office,43.653907,-79.386764,ON
5,Ontario College of Art and Design University (...,University,43.652803,-79.391074,ON
6,438 University Ave,Office,43.654296,-79.388415,ON
7,505 University Ave.,Government Building,43.656177,-79.388549,ON
8,393 University Ave,Office,43.653752,-79.388150,ON
9,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,ON
10,Queen & University,Intersection,43.650833,-79.386858,ON


## Search for Colleges 


In [35]:
# search for Cafeteria
search_query = 'College'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5CESAO5DM52GKIKYSVYSCQV4XRYXXCSFLOW50QYFJCRDJJK4&client_secret=0K4RLJWSFLTZ0CQMJOQYFGCVXQPT04JULPK4AOFC45NRRTGZ&ll=43.653963,-79.387207&v=20180604&query=College&radius=10000&limit=30'

In [36]:
# Send the GET Request and examine the results
Cresults = requests.get(url).json()
#cresults

In [37]:
# assign relevant part of JSON to venues
venues5 = Cresults['response']['venues']

# tranform venues into a dataframe
College_dataframe = json_normalize(venues5)
College_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b68ed27f964a52086932be3,444 Yonge St,CA,Toronto,Canada,at College St,860,"[444 Yonge St (at College St), Toronto ON M5B ...","[{'label': 'display', 'lat': 43.66123664500852...",43.661237,-79.383603,NaN,M5B 2H4,ON,College Park,v-1565154721,NaN
1,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4ae1c67af964a520488721e3,448 Yonge St,CA,Toronto,Canada,under College St,891,"[448 Yonge St (under College St), Toronto ON M...","[{'label': 'display', 'lat': 43.66131079616929...",43.661311,-79.382819,NaN,M5G 2K2,ON,College Subway Station,v-1565154721,NaN
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4af0615cf964a5208cdb21e3,76 Grenville St.,CA,Toronto,Canada,at Bay St.,838,"[76 Grenville St. (at Bay St.), Toronto ON M5S...","[{'label': 'display', 'lat': 43.66149084521543...",43.661491,-79.387602,NaN,M5S 1B2,ON,Women's College Hospital,v-1565154721,NaN
3,"[{'id': '4bf58dd8d48988d1a2941735', 'name': 'C...",False,4b60cb0ef964a520dbf929e3,51 Dockside Dr,CA,Toronto,Canada,George,2001,"[51 Dockside Dr (George), Toronto ON M5A 0B6, ...","[{'label': 'display', 'lat': 43.6441628, 'lng'...",43.644163,-79.366375,NaN,M5A 0B6,ON,George Brown College St. James Campus,v-1565154721,NaN
4,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",False,4ad4c064f964a52065f820e3,100 McCaul St,CA,Toronto,Canada,at Dundas St W,337,"[100 McCaul St (at Dundas St W), Toronto ON M5...","[{'label': 'display', 'lat': 43.65280251171013...",43.652803,-79.391074,NaN,M5T 1W1,ON,Ontario College of Art and Design University (...,v-1565154721,NaN


## Clean College Dataframe

In [38]:
# keep only columns that include venue name, and anything that is associated with location
College_clean_columns = ['name', 'categories'] + [col for col in College_dataframe.columns if col.startswith('location.')]+ ['id']
clean_College_dataframe = College_dataframe.loc[:,College_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_College_dataframe['categories'] = clean_College_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_College_dataframe.columns = [column.split('.')[-1] for column in clean_College_dataframe.columns]

clean_College_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,College Park,Shopping Mall,444 Yonge St,CA,Toronto,Canada,at College St,860,"[444 Yonge St (at College St), Toronto ON M5B ...","[{'label': 'display', 'lat': 43.66123664500852...",43.661237,-79.383603,NaN,M5B 2H4,ON,4b68ed27f964a52086932be3
1,College Subway Station,Metro Station,448 Yonge St,CA,Toronto,Canada,under College St,891,"[448 Yonge St (under College St), Toronto ON M...","[{'label': 'display', 'lat': 43.66131079616929...",43.661311,-79.382819,NaN,M5G 2K2,ON,4ae1c67af964a520488721e3
2,Women's College Hospital,Hospital,76 Grenville St.,CA,Toronto,Canada,at Bay St.,838,"[76 Grenville St. (at Bay St.), Toronto ON M5S...","[{'label': 'display', 'lat': 43.66149084521543...",43.661491,-79.387602,NaN,M5S 1B2,ON,4af0615cf964a5208cdb21e3
3,George Brown College St. James Campus,Community College,51 Dockside Dr,CA,Toronto,Canada,George,2001,"[51 Dockside Dr (George), Toronto ON M5A 0B6, ...","[{'label': 'display', 'lat': 43.6441628, 'lng'...",43.644163,-79.366375,NaN,M5A 0B6,ON,4b60cb0ef964a520dbf929e3
4,Ontario College of Art and Design University (...,University,100 McCaul St,CA,Toronto,Canada,at Dundas St W,337,"[100 McCaul St (at Dundas St W), Toronto ON M5...","[{'label': 'display', 'lat': 43.65280251171013...",43.652803,-79.391074,NaN,M5T 1W1,ON,4ad4c064f964a52065f820e3


In [39]:
# delete unnecessary columns
clean_College_dataframe2= clean_College_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood','address','postalCode', 'id'], axis=1)
clean_College_dataframe2

,name,categories,lat,lng,state
0,College Park,Shopping Mall,43.661237,-79.383603,ON
1,College Subway Station,Metro Station,43.661311,-79.382819,ON
2,Women's College Hospital,Hospital,43.661491,-79.387602,ON
3,George Brown College St. James Campus,Community College,43.644163,-79.366375,ON
4,Ontario College of Art and Design University (...,University,43.652803,-79.391074,ON
5,Crunch Fitness - College Park,Gym / Fitness Center,43.659263,-79.382632,ON
6,Residences at College Park North,Residential Building (Apartment / Condo),43.659822,-79.385159,ON
7,Yonge & College,Intersection,43.661371,-79.383184,ON
8,College & St George,Intersection,43.659343,-79.394475,NaN
9,College & Spadina,Road,43.657525,-79.399772,ON


In [40]:
# delete rows with none values
df_College = clean_College_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_College

,name,categories,lat,lng,state
0,College Park,Shopping Mall,43.661237,-79.383603,ON
1,College Subway Station,Metro Station,43.661311,-79.382819,ON
2,Women's College Hospital,Hospital,43.661491,-79.387602,ON
3,George Brown College St. James Campus,Community College,43.644163,-79.366375,ON
4,Ontario College of Art and Design University (...,University,43.652803,-79.391074,ON
5,Crunch Fitness - College Park,Gym / Fitness Center,43.659263,-79.382632,ON
6,Residences at College Park North,Residential Building (Apartment / Condo),43.659822,-79.385159,ON
7,Yonge & College,Intersection,43.661371,-79.383184,ON
9,College & Spadina,Road,43.657525,-79.399772,ON
10,"45 Willcocks, New College",College Residence Hall,43.661317,-79.401146,ON


In [41]:
df_College1= df_College[df_College['categories']=='College Academic Building']
df_College1

,name,categories,lat,lng,state
11,Knox College,College Academic Building,43.661561,-79.396528,ON


## Put it all into a dataframe

In [42]:
# create dataframe for all of it
dfall = pd.concat([df_College1, df_University, df_Library,df_events1,df_park], ignore_index=True)
dfall.head()

,name,categories,lat,lng,state
0,Knox College,College Academic Building,43.661561,-79.396528,ON
1,University of Toronto,University,43.662493,-79.395220,ON
2,University Avenue Plaza,Plaza,43.652695,-79.387218,ON
3,480 University Ave,Office,43.655099,-79.388648,ON
4,University Centre,Office,43.653907,-79.386764,ON


In [49]:
dfall1 =dfall

## Map the Findings

In [46]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng in zip(dfall['lat'], dfall['lng']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

## KNN

In [50]:
# prepare the dataframe 
dfall2 = dfall1.drop(['state','categories','name'], axis=1)

In [51]:
# set number of clusters
kclusters = 3


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfall2)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon in zip(dfall2['lat'], dfall2['lng']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Explore the Different Clusters 

In [66]:
dfall.insert(0, 'Cluster Labels', kmeans.labels_)
dfall2.insert(0, 'Cluster Labels', kmeans.labels_)

In [61]:
dfall2.loc[dfall2['Cluster Labels'] == 0, dfall2.columns[[1] + list(range(5, dfall2.shape[1]))]]

,lat
0,43.661561
1,43.662493
13,43.660097
15,43.665711
16,43.659280
20,43.662695
25,43.663962
26,43.663605
27,43.663331


In [62]:
dfall2.loc[dfall2['Cluster Labels'] == 2, dfall2.columns[[1] + list(range(5, dfall2.shape[1]))]]

,lat
11,43.657935
14,43.657500
18,43.659307
22,43.652981
23,43.659746


In [63]:
dfall2.loc[dfall2['Cluster Labels'] == 1, dfall2.columns[[1] + list(range(5, dfall2.shape[1]))]]

,lat
2,43.652695
3,43.655099
4,43.653907
5,43.652803
6,43.654296
7,43.656177
8,43.653752
9,43.654196
10,43.650833
12,43.655428


In [67]:
dfall.sort_values('Cluster Labels', ascending=True)

,Cluster Labels,name,categories,lat,lng,state
0,0,Knox College,College Academic Building,43.661561,-79.396528,ON
16,0,University Eye Clinic,Doctor's Office,43.659280,-79.390281,ON
15,0,University at St. Michael's College,University,43.665711,-79.389909,ON
25,0,University College Art Centre,Art Gallery,43.663962,-79.395385,ON
13,0,University of Toronto Engineering Society,Student Center,43.660097,-79.394644,ON
26,0,University of Toronto Arts Centre,Art Gallery,43.663605,-79.395159,ON
27,0,University College Residence Office,University,43.663331,-79.397947,ON
20,0,University College,University,43.662695,-79.396061,ON
1,0,University of Toronto,University,43.662493,-79.395220,ON
30,1,LSUC Library,College Library,43.651884,-79.385499,ON
